<a href="https://colab.research.google.com/github/priyanshu7466/Python/blob/main/YunSolutions/STAT_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def fetch_straddle_data(index, expiry_date):
    url = f"https://live.markethound.in/api/history/decay?name={index}&expiry={expiry_date}&dte=0"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data)
    else:
        print("Error fetching data")
        return None


index = "FINNIFTY"
expiry_date = "2024-03-14T00:00:00.000Z"
straddle_data = fetch_straddle_data(index, expiry_date)
straddle_data.head()


,status,message,result
0,success,Expiries retrieved successfully.,"{'_id': '65f043e5301dfda3b6463971', 'daysToExp..."
1,success,Expiries retrieved successfully.,"{'_id': '65e70d796f2fdc852f723066', 'daysToExp..."
2,success,Expiries retrieved successfully.,"{'_id': '65ddd17c1a3e664783907ff0', 'daysToExp..."
3,success,Expiries retrieved successfully.,"{'_id': '65d498e81a3e664783d75308', 'daysToExp..."
4,success,Expiries retrieved successfully.,"{'_id': '65cb5c132f6ea590e695fd4f', 'daysToExp..."


In [8]:
df = pd.DataFrame(straddle_data)

df['Straddle_Open'] = df['result'].apply(lambda x: x['open'])
df['Straddle_High'] = df['result'].apply(lambda x: x['high'])
df['Straddle_Low'] = df['result'].apply(lambda x: x['low'])
df['Straddle_Close'] = df['result'].apply(lambda x: x['close'])
df['Index_Open'] = df['result'].apply(lambda x: x['index_open'])
df['Index_High'] = df['result'].apply(lambda x: x['index_high'])
df['Index_Low'] = df['result'].apply(lambda x: x['index_low'])
df['Index_Close'] = df['result'].apply(lambda x: x['index_close'])
df['Date'] = df['result'].apply(lambda x: x['date'])

df.drop(['status', 'message', 'result'], axis=1, inplace=True)

df = df[['Date', 'Straddle_Open', 'Straddle_High', 'Straddle_Low', 'Straddle_Close', 'Index_Open', 'Index_High', 'Index_Low', 'Index_Close']]

print(df)


                       Date Straddle_Open Straddle_High Straddle_Low  \
0  2024-03-12T00:00:00.000Z        100.70        171.00         2.00   
1  2024-03-05T00:00:00.000Z        116.75        117.95         3.75   
2  2024-02-27T00:00:00.000Z        138.25        152.45         2.15   
3  2024-02-20T00:00:00.000Z        111.05        143.60         1.25   
4  2024-02-13T00:00:00.000Z        121.40        132.85         5.35   

  Straddle_Close Index_Open Index_High Index_Low Index_Close  
0           2.30   20847.15   21054.35  20682.35    20899.75  
1           4.40   20842.10   20951.55  20822.10    20858.10  
2           2.70   20579.10   20626.10  20535.00    20564.90  
3           1.25   20496.35   20815.20  20494.75    20812.45  
4           9.90   19961.30   20267.85  19935.85    20151.30  


In [29]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
if df is not None:

    X = df.drop(['Date', 'Straddle_Open', 'Straddle_High', 'Straddle_Low', 'Straddle_Close'], axis=1)
    y = df[['Straddle_Open', 'Straddle_High', 'Straddle_Low', 'Straddle_Close']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = MultiOutputRegressor(SVR(kernel='rbf', C=10, gamma='auto'))
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    print("Mean Squared Error:", mse)

else:
    print("Error: Failed to fetch straddle data.")

Mean Squared Error: 226.4760156249996


In [30]:
X_test.shape, y_test.shape

((1, 4), (1, 4))

In [31]:
predictions

array([[116.225 , 148.025 ,   2.7375,   4.0875]])

In [34]:
y_test

,Straddle_Open,Straddle_High,Straddle_Low,Straddle_Close
1,116.75,117.95,3.75,4.40


In [39]:
model.score(X,y)

0.7461629094567124